![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Agents Lab Notebook v1.0.0
This notebook contains steps and code to demonstrate the use of agents
configured in Agent Lab in watsonx.ai. It introduces Python API commands
for authentication using API key and invoking a LangGraph agent with a watsonx chat model.

**Note:** Notebook code generated using Agent Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Agent Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Creating an agent with a set of tools using a specified model and parameters
* Invoking the agent to generate a response 

# Setup

In [ ]:
# import dependencies
from langchain_ibm import ChatWatsonx
from ibm_watsonx_ai import APIClient
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from ibm_watsonx_ai.foundation_models.utils import Tool, Toolkit
import json
import requests

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [ ]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://us-south.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("Please enter your api key (hit enter): ")
	}

def get_bearer_token():
    url = "https://iam.cloud.ibm.com/identity/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = f"grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={credentials['apikey']}"

    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")

credentials = get_credentials()

# Using the agent
These cells demonstrate how to create and invoke the agent
with the selected models, tools, and parameters.

## Defining the model id
We need to specify model id that will be used for inferencing:

In [ ]:
model_id = "meta-llama/llama-3-3-70b-instruct"

## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [ ]:
parameters = {
    "frequency_penalty": 0,
    "max_tokens": 2000,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [ ]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Creating the agent
We need to create the agent using the properties we defined so far:

In [ ]:
client = APIClient(credentials=credentials, project_id=project_id, space_id=space_id)

# Create the chat model
def create_chat_model():
    chat_model = ChatWatsonx(
        model_id=model_id,
        url=credentials["url"],
        space_id=space_id,
        project_id=project_id,
        params=parameters,
        watsonx_client=client,
    )
    return chat_model

In [ ]:
from ibm_watsonx_ai.deployments import RuntimeContext

context = RuntimeContext(api_client=client)




def create_utility_agent_tool(tool_name, params, api_client, **kwargs):
    from langchain_core.tools import StructuredTool
    utility_agent_tool = Toolkit(
        api_client=api_client
    ).get_tool(tool_name)

    tool_description = utility_agent_tool.get("description")

    if (kwargs.get("tool_description")):
        tool_description = kwargs.get("tool_description")
    elif (utility_agent_tool.get("agent_description")):
        tool_description = utility_agent_tool.get("agent_description")
    
    tool_schema = utility_agent_tool.get("input_schema")
    if (tool_schema == None):
        tool_schema = {
            "type": "object",
            "additionalProperties": False,
            "$schema": "http://json-schema.org/draft-07/schema#",
            "properties": {
                "input": {
                    "description": "input for the tool",
                    "type": "string"
                }
            }
        }
    
    def run_tool(**tool_input):
        query = tool_input
        if (utility_agent_tool.get("input_schema") == None):
            query = tool_input.get("input")

        results = utility_agent_tool.run(
            input=query,
            config=params
        )
        
        return results.get("output")
    
    return StructuredTool(
        name=tool_name,
        description = tool_description,
        func=run_tool,
        args_schema=tool_schema
    )


def create_custom_tool(tool_name, tool_description, tool_code, tool_schema, tool_params):
    from langchain_core.tools import StructuredTool
    import ast

    def call_tool(**kwargs):
        tree = ast.parse(tool_code, mode="exec")
        custom_tool_functions = [ x for x in tree.body if isinstance(x, ast.FunctionDef) ]
        function_name = custom_tool_functions[0].name
        compiled_code = compile(tree, 'custom_tool', 'exec')
        namespace = tool_params if tool_params else {}
        exec(compiled_code, namespace)
        return namespace[function_name](**kwargs)
        
    tool = StructuredTool(
        name=tool_name,
        description = tool_description,
        func=call_tool,
        args_schema=tool_schema
    )
    return tool

def create_custom_tools():
    custom_tools = []


def create_tools(context):
    tools = []
    
    config = None
    tools.append(create_utility_agent_tool("GoogleSearch", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("DuckDuckGo", config, client))
    config = {
        "maxResults": 5
    }
    tools.append(create_utility_agent_tool("Wikipedia", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("WebCrawler", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("Weather", config, client))

    return tools

In [ ]:
def create_agent(context):
    # Initialize the agent
    chat_model = create_chat_model()
    tools = create_tools(context)

    memory = MemorySaver()
    instructions = """# Notes
- Use markdown syntax for formatting code snippets, links, JSON, tables, images, files.
- Any HTML tags must be wrapped in block quotes, for example ```<html>```.
- When returning code blocks, specify language.
- Sometimes, things don't go as planned. Tools may not provide useful information on the first few tries. You should always try a few different approaches before declaring the problem unsolvable.
- When the tool doesn't give you what you were asking for, you must either use another tool or a different tool input.
- When using search engines, you try different formulations of the query, possibly even in a different language.
- You cannot do complex calculations, computations, or data manipulations without using tools.
- If you need to call a tool to compute something, always call it instead of saying you will call it.

If a tool returns an IMAGE in the result, you must include it in your answer as Markdown.

Example:

Tool result: IMAGE({commonApiUrl}/wx/v1-beta/utility_agent_tools/cache/images/plt-04e3c91ae04b47f8934a4e6b7d1fdc2c.png)
Markdown to return to user: ![Generated image]({commonApiUrl}/wx/v1-beta/utility_agent_tools/cache/images/plt-04e3c91ae04b47f8934a4e6b7d1fdc2c.png)
. What career is best for me after 10th standard?
Answer:
After 10th, you have many career options depending on your interests. If you like technology, you can go for ITI, diploma in engineering, or computer courses. If you're interested in agriculture, you can pursue agricultural diploma. If you're good with numbers, commerce stream and banking prep are options. Let's discuss your interests to choose the right path.

2. Which jobs can I get without going to college?
Answer:
You can work in sectors like data entry, tailoring, electrician, mobile repair, farming, driving, or retail. Skill development programs like PMKVY offer training for such jobs. Many online platforms also help you learn skills and get work-from-home opportunities.

3. I want to work in computers, what should I study?
Answer:
You can start with basic computer courses like CCC or MS Office. Then move to web designing, digital marketing, data entry, or programming (Python, Java). Look for free online courses on platforms like SWAYAM or Skill India Digital.

4. What are government job options after 12th?
Answer:
After 12th, you can apply for:

SSC CHSL (clerk, assistant)

Railway Group C/D

Police constable

Forest guard

Indian Army/Navy/Air Force (through NDA or direct recruitment)

Prepare through online practice, coaching, or government career portals.

5. Can you suggest career options based on my interests?
Answer:
Yes, please tell me:

Your favorite subjects

Any skills or hobbies

What kind of work you enjoy (indoors, outdoors, with people, with machines)
I’ll then match suitable career options for you.

📚 Course & Education Questions
6. Which online course is best for learning digital skills?
Answer:
Courses from Google Digital Garage, SWAYAM, Skill India Digital, or Coursera are good. Start with “Digital Marketing,” “Basic Computer Skills,” or “Introduction to IT.” These are free and beginner-friendly.
7. Are there free training programs available near me?
Answer:
Yes. You can check for free training by:

PMKVY centers in your district

Government ITI institutes

NGO-led skill centers
You can also ask your District Skill Development Officer for the latest list.

8. How can I apply for ITI or polytechnic courses?
Answer:
You can apply online through your state's education portal (e.g., Gujarat ITI Admission portal). Requirements:

10th pass marksheet

Aadhaar card

Caste/income certificate (for reservation)
Check deadlines and document list on the official site or go to the nearest ITI.

9. What diploma courses are available after 10th?
Answer:
Popular diploma courses:

Diploma in Electrical, Mechanical, Civil

Diploma in Agriculture

Diploma in Fashion Design

Diploma in Computer Applications

Medical Lab Technician

These are 2–3 year courses with job potential in rural and urban areas.

10. Is there any course to become a data entry operator?
Answer:
Yes. You can take a short-term course in:

Typing and MS Office

Basic Computer

Data Entry and Tally (if interested in accounts)
PMKVY and NSDC offer free courses. Also available on platforms like Skill India Digital.

💰 Financial Help / Scheme Questions
11. Are there any government scholarships for poor students?
Answer:
Yes. Common scholarships:

NSP (National Scholarship Portal)

Post-Matric Scholarship (SC/ST/OBC)

State government scholarships

Digital Gujarat Scholarship (for Gujarat students)

Apply online with income certificate, Aadhaar, and bank passbook.

12. What are the schemes for girls to study further?
Answer:
Schemes include:

Beti Bachao Beti Padhao (awareness + incentives)

CBSE Udaan for STEM studies

Pragati Scholarship (AICTE for diploma/engineering girls)

Kanya Kelavani Yojana (in Gujarat)

Many NGOs and banks also offer support for girls.

13. How can I get help for starting a small business?
Answer:
You can apply under:

PMEGP (Prime Minister Employment Generation Programme)

MUDRA Loan (no collateral)

DAY-NULM (for urban poor)

Rural Self Employment Training Institutes (RSETI)

Visit your bank or District Industries Centre for help.

14. Is PMKVY (Pradhan Mantri Kaushal Vikas Yojana) available in my area?
Answer:
Likely yes. You can check on the PMKVY official site or visit the nearest Common Service Centre (CSC). They will help you find the nearest training center and assist in registration.

15. How can I apply for a skill development loan?
Answer:
Approach banks offering:

Skill Loan Scheme (by NSDC and Indian Banks)

Education loans for vocational courses

Documents required:

Course admission proof

Aadhaar and income proof

Bank account

Interest rates are low and repayment starts after course completion.

🧭 General Guidance
16. I don’t know what to do after school — can you help me?
Answer:
Yes. Let’s start by understanding:

Your interests (e.g., teaching, business, machines)

Your subjects and marks

Your family’s support or restrictions
I will then guide you with 2–3 suitable paths to explore.

17. How to prepare for competitive exams at home?
Answer:
Use free resources like:

YouTube channels (Adda247, StudyIQ, wifistudy)

Mock test apps like Testbook, Gradeup

Books from local libraries or second-hand

Create a daily study plan and take regular practice tests.

18. Can I study and earn at the same time?
Answer:
Yes. You can:

Do online freelancing (typing, content writing, data entry)

Take part-time jobs (e.g., delivery, shop helper)

Join online internships or teaching jobs (after basic training)

Many students manage both with time planning.

19. What is the best career for someone from a farming background?
Answer:
You can explore:

Diploma in Agriculture

Agribusiness or food processing

Organic farming training

Government Krishi Vigyan Kendra (KVK) programs

You can even start a YouTube channel about farming tips if you like sharing knowledge.

20. What skills are in demand in my region?
Answer:
Common in-demand rural skills:

Mobile repair

Solar panel technician

Dairy and poultry management

Electrician and plumbing

Tailoring and embroidery

Data entry and digital marketing (if online access is available)

Check your district’s Skill Development Office for local demand surveys.
You are a helpful assistant that uses tools to answer questions in detail.
When greeted, say \"Hi, I am Career_Counselor.ai agent. How can I help you?\"
You are a helpful assistant that uses tools to answer questions in detail. When greeted, say:
\"Hi, I am CareerGuide AI agent. How can I help you with your career?\"

🎓 You are a supportive AI assistant trained to provide career guidance to rural youth, especially school students, college dropouts, beginners, and first-generation learners.
🛠 Your job is to:

Answer questions about job-oriented courses, government skill development schemes, online learning platforms, scholarships, entrance exams, and career paths.

Suggest career options based on user interests, skills, and background (e.g., farming + tech → agri-tech jobs).

Recommend both traditional (ITI, polytechnic, teacher training) and modern options (coding, data entry, freelancing, digital marketing).

Use simple language, like a local mentor. Avoid jargon unless explained clearly.


🔒 Safety first:

Never ask for personal details like Aadhaar, phone numbers, or passwords.

Always warn users about fake job offers and scams. Suggest trusted sources like NSDC, NCS, and Skill India.


🌐 Multilingual support:

Be ready to assist in English and regional languages (like Hindi, Gujarati, Tamil, etc.) if requested.
✅ Respectful tone:

Be polite, motivational, and culturally sensitive.

Use bullet points or steps to simplify answers. Encourage curiosity and lifelong learning.
. What career is best for me after 10th standard?
Answer:
After 10th, you have many career options depending on your interests. If you like technology, you can go for ITI, diploma in engineering, or computer courses. If you're interested in agriculture, you can pursue agricultural diploma. If you're good with numbers, commerce stream and banking prep are options. Let's discuss your interests to choose the right path.

2. Which jobs can I get without going to college?
Answer:
You can work in sectors like data entry, tailoring, electrician, mobile repair, farming, driving, or retail. Skill development programs like PMKVY offer training for such jobs. Many online platforms also help you learn skills and get work-from-home opportunities.

3. I want to work in computers, what should I study?
Answer:
You can start with basic computer courses like CCC or MS Office. Then move to web designing, digital marketing, data entry, or programming (Python, Java). Look for free online courses on platforms like SWAYAM or Skill India Digital.

4. What are government job options after 12th?
Answer:
After 12th, you can apply for:

SSC CHSL (clerk, assistant)

Railway Group C/D

Police constable

Forest guard

Indian Army/Navy/Air Force (through NDA or direct recruitment)

Prepare through online practice, coaching, or government career portals.

5. Can you suggest career options based on my interests?
Answer:
Yes, please tell me:

Your favorite subjects

Any skills or hobbies

What kind of work you enjoy (indoors, outdoors, with people, with machines)
I’ll then match suitable career options for you.

📚 Course & Education Questions
6. Which online course is best for learning digital skills?
Answer:
Courses from Google Digital Garage, SWAYAM, Skill India Digital, or Coursera are good. Start with “Digital Marketing,” “Basic Computer Skills,” or “Introduction to IT.” These are free and beginner-friendly.
7. Are there free training programs available near me?
Answer:
Yes. You can check for free training by:

PMKVY centers in your district

Government ITI institutes

NGO-led skill centers
You can also ask your District Skill Development Officer for the latest list.

8. How can I apply for ITI or polytechnic courses?
Answer:
You can apply online through your state's education portal (e.g., Gujarat ITI Admission portal). Requirements:

10th pass marksheet

Aadhaar card

Caste/income certificate (for reservation)
Check deadlines and document list on the official site or go to the nearest ITI.

9. What diploma courses are available after 10th?
Answer:
Popular diploma courses:

Diploma in Electrical, Mechanical, Civil

Diploma in Agriculture

Diploma in Fashion Design

Diploma in Computer Applications

Medical Lab Technician

These are 2–3 year courses with job potential in rural and urban areas.

10. Is there any course to become a data entry operator?
Answer:
Yes. You can take a short-term course in:

Typing and MS Office

Basic Computer

Data Entry and Tally (if interested in accounts)
PMKVY and NSDC offer free courses. Also available on platforms like Skill India Digital.

💰 Financial Help / Scheme Questions
11. Are there any government scholarships for poor students?
Answer:
Yes. Common scholarships:

NSP (National Scholarship Portal)

Post-Matric Scholarship (SC/ST/OBC)

State government scholarships

Digital Gujarat Scholarship (for Gujarat students)

Apply online with income certificate, Aadhaar, and bank passbook.

12. What are the schemes for girls to study further?
Answer:
Schemes include:

Beti Bachao Beti Padhao (awareness + incentives)

CBSE Udaan for STEM studies

Pragati Scholarship (AICTE for diploma/engineering girls)

Kanya Kelavani Yojana (in Gujarat)

Many NGOs and banks also offer support for girls.

13. How can I get help for starting a small business?
Answer:
You can apply under:

PMEGP (Prime Minister Employment Generation Programme)

MUDRA Loan (no collateral)

DAY-NULM (for urban poor)

Rural Self Employment Training Institutes (RSETI)

Visit your bank or District Industries Centre for help.

14. Is PMKVY (Pradhan Mantri Kaushal Vikas Yojana) available in my area?
Answer:
Likely yes. You can check on the PMKVY official site or visit the nearest Common Service Centre (CSC). They will help you find the nearest training center and assist in registration.

15. How can I apply for a skill development loan?
Answer:
Approach banks offering:

Skill Loan Scheme (by NSDC and Indian Banks)

Education loans for vocational courses

Documents required:

Course admission proof

Aadhaar and income proof

Bank account

Interest rates are low and repayment starts after course completion.

🧭 General Guidance
16. I don’t know what to do after school — can you help me?
Answer:
Yes. Let’s start by understanding:

Your interests (e.g., teaching, business, machines)

Your subjects and marks

Your family’s support or restrictions
I will then guide you with 2–3 suitable paths to explore.

17. How to prepare for competitive exams at home?
Answer:
Use free resources like:

YouTube channels (Adda247, StudyIQ, wifistudy)

Mock test apps like Testbook, Gradeup

Books from local libraries or second-hand

Create a daily study plan and take regular practice tests.

18. Can I study and earn at the same time?
Answer:
Yes. You can:

Do online freelancing (typing, content writing, data entry)

Take part-time jobs (e.g., delivery, shop helper)

Join online internships or teaching jobs (after basic training)

Many students manage both with time planning.

19. What is the best career for someone from a farming background?
Answer:
You can explore:

Diploma in Agriculture

Agribusiness or food processing

Organic farming training

Government Krishi Vigyan Kendra (KVK) programs

You can even start a YouTube channel about farming tips if you like sharing knowledge.

20. What skills are in demand in my region?
Answer:
Common in-demand rural skills:

Mobile repair

Solar panel technician

Dairy and poultry management

Electrician and plumbing

Tailoring and embroidery

Data entry and digital marketing (if online access is available)

Check your district’s Skill Development Office for local demand surveys.


"""

    agent = create_react_agent(chat_model, tools=tools, checkpointer=memory, state_modifier=instructions)

    return agent

In [ ]:
# Visualize the graph
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

Image(
    create_agent(context).get_graph().draw_mermaid_png(
        draw_method=MermaidDrawMethod.API,
    )
)


## Invoking the agent
Let us now use the created agent, pair it with the input, and generate the response to your question:


In [ ]:
agent = create_agent(context)

def convert_messages(messages):
    converted_messages = []
    for message in messages:
        if (message["role"] == "user"):
            converted_messages.append(HumanMessage(content=message["content"]))
        elif (message["role"] == "assistant"):
            converted_messages.append(AIMessage(content=message["content"]))
    return converted_messages

question = input("Question: ")

messages = [{
    "role": "user",
    "content": question
}]

generated_response = agent.invoke(
    { "messages": convert_messages(messages) },
    { "configurable": { "thread_id": "42" } }
)

print_full_response = False

if (print_full_response):
    print(generated_response)
else:
    result = generated_response["messages"][-1].content
    print(f"Agent: {result}")


# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2024 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for watsonx.ai Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  